In [ ]:
import cftools as cft
import xarray as xr
import numpy as np
import cftools.matplotlib as mpl
from cftools.matplotlib import plt
import cartopy
import cartopy.crs as ccrs
from shapely.geometry import mapping, Polygon
import fiona
import os

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
DATA = 'in/era_interim_waves_height_1979_2017.nc'

In [ ]:
waves_height = cft.io.open_dataarray(DATA)

In [ ]:
mask = waves_height > 3.6576

In [ ]:
mask_grouped = mask.groupby('time.month')

In [ ]:
def plot_PlateCarree(data, lev=np.arange(30,80,20), cmap='red'):
    plt.figure(figsize=[30,10])
    
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    ax.add_feature(
        cartopy.feature.LAND,
        zorder=100, 
        edgecolor=[0.2,0.2,0.2], 
        facecolor=[0.5,0.5,0.5])
    
    ax.coastlines()
          
    #data.plot.imshow(interpolation='bilinear', cmap='coolwarm')
    cs = data.plot.contour(levels = lev, transform=ccrs.PlateCarree(), cmap=cmap, linewidths=1) # cs stand for 'contour set'
    plt.clabel(cs, inline=False, fontsize=10)
    plt.title('Percentage of waves with height > 12 ft')
    
    return cs
    
    # Alternative plot method
    # pl = plt.contourf(data.lon, data.lat, data, lev, transform=ccrs.PlateCarree(), cmap=cmap)
    # cbar = plt.colorbar(pl)
    
    
for month_index in range(1,2):
    samples = len(mask_grouped.groups[month_index])
    over_thresh = mask_grouped.reduce(np.count_nonzero, dim='time').sel(month=month_index)
    over_thresh_percentage = over_thresh/samples*100
    cs = plot_PlateCarree(over_thresh_percentage)

In [ ]:
isolines = [] # a list of lists containing tuples
for contour in cs.collections:
    for path in contour.get_paths():
        isolines.append(list(map(tuple, path.vertices)))

polygons = [] # a list of shapely Polygons objects
for path in isolines:
    polygons.append(Polygon(path))

In [ ]:
# create output directory for shapefiles
path = os.path.dirname(os.path.abspath('__file__'))
path = os.path.abspath(os.path.join(path,'../out/shp'))

if not os.path.exists(path):
   os.makedirs(path)

In [ ]:
# Define a polygon feature geometry with one attribute
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int'},
}

# Write a new Shapefile
with fiona.open('../out/shp/waves-height-isolines.shp', 'w', 'ESRI Shapefile', schema) as shapefile:
    for poly in polygons:
        shapefile.write({
            'geometry': mapping(poly),
            'properties': {'id': 123},
        })